In [ ]:
!pip install -Uquiet langchain_ollama
!ollama pull llama3
!pip install aperturedb
!adb config create local --active --no-interactive

In [1]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3"
)

In [2]:
input_text = "The meaning of life is 42"
vector = embed.embed_query(input_text)
print(type(vector))
print(len(vector))
print(vector[:3])


<class 'list'>
4096
[0.016020482, 0.013659676, -0.008259736]


In [7]:
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain framework is used to create context aware reasoning applications."
vectorstore = InMemoryVectorStore.from_texts([text], embedding=embed)
retriver = vectorstore.as_retriever()
resp = retriver.invoke("What is LangChain")
print(resp[0].page_content[:20])

LangChain framework 


In [38]:
vectorstore._collection.count()

AttributeError: 'ApertureDB' object has no attribute '_collection'

In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.aperturedata.io")
docs = loader.load()
print(len(docs))

1


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()
documents = splitter.split_documents(docs)
print(len(documents))

1


In [15]:
from langchain_community.vectorstores import ApertureDB

In [36]:
!adb config create local --active --no-interactive --port 55556 --overwrite

In [37]:
vectorstore = ApertureDB.from_documents(documents, embedding=embed)

Progress:   0%|                                                                               | 0.00/1.00 [00:02<?, ?items/s]


In [39]:
adb_retriever = vectorstore.as_retriever()
resp = adb_retriever.invoke("ApertureDB")

In [ ]:
print(resp[0].page_content)

In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import create_retrieval_chain

template = """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.2")
model_chain = prompt | model
chain = create_retrieval_chain(adb_retriever, model_chain)

In [68]:
resp = chain.invoke({"input": "what is ApertureDB?"})
print(resp['answer'])

According to the provided context, ApertureDB appears to be a database that supports graph and vector search over multimodal data. It offers various resources, including documentation, quick start guides, query references, command line interfaces, and Python SDKs, for helping users work with the database.


In [70]:
resp = chain.invoke({"input": "what is graph? And by the way Avery Lancaster is the CEO of Insurellm"})
print(resp['answer'])

I can't verify that information about Avery Lancaster. Can I help you with something else regarding the context provided?


In [71]:
resp = chain.invoke({"input": "who is Lancaster?"})
print(resp['answer'])

There is no mention of "Lancaster" in the provided context.


In [72]:
print(resp["context"])

[Document(id='906b0f6d-7f7d-4fbe-a8d8-24f59dcabaf0', metadata={'language': 'en', 'source': 'https://docs.aperturedata.io', 'title': "ApertureDB's Documentation | ApertureDB"}, page_content="ApertureDB's Documentation | ApertureDB\n\n\n\n\n\n\n\n\n\n\n\nSkip to main contentApertureDBQuick StartQuery ReferenceCommand LineFAQsTroubleshootingGitHubApertureDataSearchApertureDB's DocumentationIntroductionDatabase ConceptsGet StartedHow to GuidesWorkflowsIntegrationsApertureDB Query LanguageApertureDB Python SDKProduction Deployment and AdministrationBenchmarks & ComparisonsApertureDB Community Projects*ApertureDB's DocumentationNew to ApertureDB?Learn about objects, commands, queries, new terminology and concepts introduced by ApertureDB for graph and vector search over multimodal data.Learn DB FundamentalsGet StartedLearn how to setup ApertureDB server with its different deployment options, configure client, and start with loading and querying a multimodal datasetQuick Start GuideApertureDB